In [1]:
from util import *
import torch
from allennlp.commands.elmo import ElmoEmbedder
from model import *

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
# parse the data
wsd_data, train_data, test_data, dev_data = parse_data()

# return the raw sentences from the EUD for train, test, and dev
# test the first 20 sentences
train_sentences, train_word_sense, train_word_index, test_sentences, test_word_sense, test_word_index, dev_sentences, dev_word_sense, dev_word_index = get_raw_sentences(wsd_data, train_data, test_data, dev_data, 20)

# ELMo setup
# ELMo is tuned to lower dimension (256) by MLP in Model
elmo = ElmoEmbedder()
model = Model(elmo_class = elmo)

# MLP illustration
print_fine_tuning_MLP(model, 'WSD')

# forward propagation
# ELMo (1024) -> dimension reduction (256) -> bi-LSTM (512) -> fine-tuning MLP (10)
model.forward(train_sentences, train_word_index)

Parsed 439312 word sense data from White et. al., 2016.
Parsed 12543 training data from UD_English-EWT/en_ewt-ud-train.conllu.
Parsed 2077 testing data from UD_English-EWT/en_ewt-ud-test.conllu
Parsed 2002 dev data from UD_English-EWT/en_ewt-ud-dev.conllu
Parsed 20 sentences
******************* Data Example ***********************
Sentence: ['on', 'August', '9', ',', '2004', ',', 'it', 'be', 'announce', 'that', 'in', 'the', 'spring', 'of', '2001', ',', 'a', 'man', 'name', 'El', '-', 'Shukrijumah', ',', 'also', 'know', 'as', 'Jafar', 'the', 'Pilot', ',', 'who', 'be', 'part', 'of', 'a', '"', 'second', 'wave', ',', '"', 'have', 'be', 'case', 'New', 'York', 'City', 'helicopter', '.']
Target Word Index: 12
Target Word Sense (index in WordNet 3.1): spring.n.01
********************************************************
******************* fine-tuning MLP structure ***********************
Current Task: WSD
Linear(in_features=512, out_features=300, bias=True)
ReLU()
Linear(in_features=300, out_fe

[tensor([0.1026, 0.0985, 0.1047, 0.1024, 0.0982, 0.0936, 0.1035, 0.0998, 0.0961,
         0.1006], grad_fn=<SoftmaxBackward>),
 tensor([0.1026, 0.0985, 0.1047, 0.1024, 0.0982, 0.0936, 0.1035, 0.0998, 0.0961,
         0.1006], grad_fn=<SoftmaxBackward>),
 tensor([0.1026, 0.0985, 0.1047, 0.1024, 0.0982, 0.0936, 0.1035, 0.0998, 0.0961,
         0.1006], grad_fn=<SoftmaxBackward>),
 tensor([0.1026, 0.0985, 0.1047, 0.1024, 0.0982, 0.0936, 0.1035, 0.0998, 0.0961,
         0.1006], grad_fn=<SoftmaxBackward>),
 tensor([0.1026, 0.0985, 0.1047, 0.1024, 0.0982, 0.0936, 0.1035, 0.0998, 0.0961,
         0.1006], grad_fn=<SoftmaxBackward>),
 tensor([0.1026, 0.0985, 0.1047, 0.1024, 0.0982, 0.0936, 0.1035, 0.0998, 0.0961,
         0.1006], grad_fn=<SoftmaxBackward>),
 tensor([0.1026, 0.0985, 0.1047, 0.1024, 0.0982, 0.0936, 0.1035, 0.0998, 0.0961,
         0.1006], grad_fn=<SoftmaxBackward>),
 tensor([0.1026, 0.0985, 0.1047, 0.1024, 0.0982, 0.0936, 0.1035, 0.0998, 0.0961,
         0.1006], grad_fn=<Sof